In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import csv
from kmodes.kprototypes import KPrototypes

In [2]:
df = pd.read_csv("./bank-additional-full.csv")

In [3]:
customer = df

1 - age (numeric)

   2 - job : type of job (categorical: "admin.","blue-collar","entrepreneur","housemaid","management","retired","self-employed","services","student","technician","unemployed","unknown")

   3 - marital : marital status (categorical: "divorced","married","single","unknown"; note: "divorced" means divorced or widowed)

   4 - education (categorical: "basic.4y","basic.6y","basic.9y","high.school","illiterate","professional.course","university.degree","unknown")

   5 - default: has credit in default? (categorical: "no","yes","unknown")

   6 - housing: has housing loan? (categorical: "no","yes","unknown")

   7 - loan: has personal loan? (categorical: "no","yes","unknown")

   # related with the last contact of the current campaign:

   8 - contact: contact communication type (categorical: "cellular","telephone") 

   9 - month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")

  10 - day_of_week: last contact day of the week (categorical: "mon","tue","wed","thu","fri")

  11 - duration: last contact duration, in seconds (numeric). Important note:  this attribute highly affects the output target (e.g., if duration=0 then y="no"). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

   # other attributes(contact behaviour):

  12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)

  13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)

  14 - previous: number of contacts performed before this campaign and for this client (numeric)

  15 - poutcome: outcome of the previous marketing campaign (categorical: "failure","nonexistent","success")

   # social and economic context attributes

  16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)(drop)

  17 - cons.price.idx: consumer price index - monthly indicator (numeric)     

  18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)     

  19 - euribor3m: euribor 3 month rate - daily indicator (numeric)(drop)

  20 - nr.employed: number of employees - quarterly indicator (numeric)(drop)



  Output variable (desired target):

  21 - y - has the client subscribed a term deposit? (binary: "yes","no")(drop)


In [4]:
# customer = customer.drop(['emp.var.rate','euribor3m','nr.employed','y','job'],axis=1)

In [5]:
customer.columns[0]

'age'

In [6]:
customer

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [7]:
for i in customer.columns:
    print(i)
    print(customer[i].value_counts())
    print("\n--------------------------------------\n")

age
31    1947
32    1846
33    1833
36    1780
35    1759
      ... 
89       2
91       2
87       1
94       1
95       1
Name: age, Length: 78, dtype: int64

--------------------------------------

job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: job, dtype: int64

--------------------------------------

marital
married     24928
single      11568
divorced     4612
unknown        80
Name: marital, dtype: int64

--------------------------------------

education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
unknown                 1731
illiterate                18
Name: education, dtype: int64

-------------------------------

In [11]:
customer = customer.drop(['y'],axis=1)

In [12]:
ind = []
l = customer.dtypes
for i in range(len(l)):
    if (str(l[i])=='object'):
        ind.append(i)

In [13]:
ind

[1, 2, 3, 4, 5, 6, 7, 8, 9, 14]

In [14]:
c_data = [customer.columns[i] for i in ind]

In [15]:
c_data

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'poutcome']

In [16]:
d={}
k=0
for i in customer.columns:
    d[i]=k
    k=k+1

In [17]:
customer.rename(columns=d,inplace=True)

In [18]:
customer

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,334,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,383,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,189,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,442,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6


In [19]:
kproto = KPrototypes(n_clusters=10, init='Cao', verbose=2)
clusters = kproto.fit_predict(customer, categorical=ind)

Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/100, moves: 19007, ncost: 613401468.7091238
Run: 1, iteration: 2/100, moves: 7088, ncost: 555433380.2564982
Run: 1, iteration: 3/100, moves: 4865, ncost: 5250

Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
In

Run: 4, iteration: 46/100, moves: 2, ncost: 430738373.50117445
Run: 4, iteration: 47/100, moves: 0, ncost: 430738373.50117445
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 5, iteration: 1/100, moves: 10357, ncost: 612932404.3936435
Run: 5, iteration: 2/100, moves: 7081, ncost: 541825463.609659
Run: 5, iteration: 3/100, moves: 5241, ncost: 507496718.75243443
Run: 5, iteration: 4/100, moves: 4529, ncost: 480927462.4659079
Run: 5, iteration: 5/100, moves: 3812, ncost: 465467964.0675693
Run: 5, iteration: 6/100, moves: 2572, ncost: 456866839.7616567
Run: 5, iteration: 7/100, moves: 1215, ncost: 452269669.68835133
Ru

Run: 6, iteration: 34/100, moves: 71, ncost: 446696138.28864646
Run: 6, iteration: 35/100, moves: 45, ncost: 446690228.2811035
Run: 6, iteration: 36/100, moves: 60, ncost: 446673154.0273455
Run: 6, iteration: 37/100, moves: 51, ncost: 446634699.9329717
Run: 6, iteration: 38/100, moves: 59, ncost: 446547894.4357548
Run: 6, iteration: 39/100, moves: 66, ncost: 446494509.92004764
Run: 6, iteration: 40/100, moves: 35, ncost: 446482259.82123965
Run: 6, iteration: 41/100, moves: 35, ncost: 446470306.0716356
Run: 6, iteration: 42/100, moves: 44, ncost: 446447474.29097784
Run: 6, iteration: 43/100, moves: 49, ncost: 446429380.1120706
Run: 6, iteration: 44/100, moves: 25, ncost: 446418505.5025684
Run: 6, iteration: 45/100, moves: 26, ncost: 446414530.5336087
Run: 6, iteration: 46/100, moves: 38, ncost: 446411390.4775934
Run: 6, iteration: 47/100, moves: 41, ncost: 446409982.37918645
Run: 6, iteration: 48/100, moves: 12, ncost: 446409741.0896145
Run: 6, iteration: 49/100, moves: 10, ncost: 44640

Run: 8, iteration: 35/100, moves: 22, ncost: 430739913.6863727
Run: 8, iteration: 36/100, moves: 18, ncost: 430738922.903483
Run: 8, iteration: 37/100, moves: 8, ncost: 430738717.5706005
Run: 8, iteration: 38/100, moves: 1, ncost: 430738679.1249638
Run: 8, iteration: 39/100, moves: 9, ncost: 430738380.0505674
Run: 8, iteration: 40/100, moves: 2, ncost: 430738373.50117505
Run: 8, iteration: 41/100, moves: 0, ncost: 430738373.50117505
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing cl

Run: 10, iteration: 51/100, moves: 60, ncost: 458703134.4880707
Run: 10, iteration: 52/100, moves: 73, ncost: 458696770.36945367
Run: 10, iteration: 53/100, moves: 35, ncost: 458694330.3045688
Run: 10, iteration: 54/100, moves: 16, ncost: 458692158.867135
Run: 10, iteration: 55/100, moves: 4, ncost: 458692096.9088961
Run: 10, iteration: 56/100, moves: 0, ncost: 458692096.9088961
Best run was number 5


In [21]:
kproto.cost_

430738373.50117177

In [22]:
clusters

array([6, 2, 6, ..., 3, 3, 3], dtype=uint16)

In [23]:
cluster_dict = {}
for i in range(0,10):
    cluster_dict[i]=[]

In [24]:
# cluster_dict = {}
for i in range(len(clusters)):
    cluster_dict[clusters[i]].append(i)

In [25]:
for i,j in cluster_dict.items():
    print(str(i)+" : "+str(len(j)))

0 : 2148
1 : 5954
2 : 14060
3 : 3872
4 : 4459
5 : 1245
6 : 8268
7 : 104
8 : 267
9 : 811


In [26]:
cluster_dict

{0: [57,
  61,
  111,
  131,
  169,
  182,
  195,
  198,
  243,
  249,
  276,
  286,
  296,
  329,
  346,
  395,
  413,
  421,
  431,
  462,
  467,
  504,
  539,
  575,
  611,
  614,
  659,
  689,
  715,
  723,
  773,
  779,
  806,
  820,
  832,
  844,
  876,
  913,
  937,
  967,
  987,
  1005,
  1020,
  1059,
  1113,
  1124,
  1129,
  1166,
  1269,
  1283,
  1299,
  1318,
  1348,
  1365,
  1368,
  1414,
  1418,
  1424,
  1470,
  1501,
  1505,
  1525,
  1546,
  1551,
  1568,
  1614,
  1624,
  1632,
  1636,
  1655,
  1662,
  1707,
  1720,
  1725,
  1763,
  1819,
  1847,
  1869,
  1938,
  1980,
  1995,
  2000,
  2039,
  2064,
  2082,
  2090,
  2097,
  2110,
  2126,
  2143,
  2260,
  2270,
  2315,
  2342,
  2393,
  2400,
  2409,
  2421,
  2431,
  2437,
  2444,
  2449,
  2454,
  2459,
  2465,
  2473,
  2475,
  2482,
  2541,
  2565,
  2567,
  2610,
  2616,
  2635,
  2638,
  2663,
  2684,
  2713,
  2733,
  2745,
  2775,
  2779,
  2814,
  2830,
  2835,
  2840,
  2841,
  2865,
  2887,
  2905,


In [27]:
kproto.cluster_centroids_

[array([[ 3.97886406e+01,  7.68675978e+02,  2.43296089e+00,
          9.99000000e+02,  9.91620112e-02,  1.63640596e-01,
          9.35927956e+01, -4.08717877e+01,  3.70223557e+00,
          5.17150582e+03],
        [ 3.91084985e+01,  1.04698354e+02,  2.30903594e+00,
          9.99000000e+02,  3.31877729e-01, -2.03380920e+00,
          9.29629392e+01, -4.33035942e+01,  1.20844071e+00,
          5.07987123e+03],
        [ 4.03502845e+01,  9.81303698e+01,  3.08157895e+00,
          9.99000000e+02,  2.99431010e-02,  1.11250356e+00,
          9.38189874e+01, -3.95413869e+01,  4.81285000e+00,
          5.21484767e+03],
        [ 3.97383781e+01,  2.76698347e+02,  1.92742769e+00,
          9.99000000e+02,  3.37035124e-01, -2.03016529e+00,
          9.30248843e+01, -4.27868285e+01,  1.17869189e+00,
          5.07329323e+03],
        [ 4.00661583e+01,  4.75797712e+02,  2.32114824e+00,
          9.99000000e+02,  9.64341781e-02,  2.31688719e-01,
          9.36139843e+01, -4.06384167e+01,  3.797440

In [35]:
for i in set(kproto.labels_):
    index = kproto.labels_ == i
    plt.plot(X[index, 0], X[index, 1], 'o')
    plt.suptitle('Data points categorized with category score', fontsize=18)
    plt.xlabel('Category Score', fontsize=16)
    plt.ylabel('Category Type', fontsize=16)
plt.show()

IndexError: boolean index did not match indexed array along dimension 0; dimension is 41189 but corresponding boolean dimension is 41188

In [32]:
syms = np.genfromtxt('bank-additional-full.csv', dtype=str, delimiter=',')[:, 1]
for s, c in zip(syms, clusters):
    print("Result: {}, cluster:{}".format(s, c))

Result: "job", cluster:6
Result: "housemaid", cluster:2
Result: "services", cluster:6
Result: "services", cluster:2
Result: "admin.", cluster:6
Result: "services", cluster:6
Result: "services", cluster:2
Result: "admin.", cluster:6
Result: "blue-collar", cluster:4
Result: "technician", cluster:2
Result: "services", cluster:2
Result: "blue-collar", cluster:6
Result: "services", cluster:2
Result: "blue-collar", cluster:6
Result: "housemaid", cluster:2
Result: "blue-collar", cluster:6
Result: "retired", cluster:6
Result: "blue-collar", cluster:4
Result: "blue-collar", cluster:6
Result: "blue-collar", cluster:6
Result: "management", cluster:2
Result: "unemployed", cluster:6
Result: "blue-collar", cluster:6
Result: "retired", cluster:6
Result: "technician", cluster:6
Result: "admin.", cluster:2
Result: "technician", cluster:2
Result: "technician", cluster:6
Result: "self-employed", cluster:6
Result: "technician", cluster:6
Result: "unknown", cluster:6
Result: "admin.", cluster:4
Result: "te

Result: "unknown", cluster:4
Result: "blue-collar", cluster:6
Result: "blue-collar", cluster:4
Result: "blue-collar", cluster:2
Result: "admin.", cluster:6
Result: "blue-collar", cluster:4
Result: "admin.", cluster:6
Result: "admin.", cluster:6
Result: "management", cluster:4
Result: "student", cluster:2
Result: "blue-collar", cluster:4
Result: "blue-collar", cluster:2
Result: "retired", cluster:4
Result: "services", cluster:4
Result: "services", cluster:0
Result: "technician", cluster:4
Result: "technician", cluster:2
Result: "technician", cluster:9
Result: "technician", cluster:4
Result: "services", cluster:2
Result: "blue-collar", cluster:4
Result: "blue-collar", cluster:2
Result: "technician", cluster:0
Result: "blue-collar", cluster:4
Result: "housemaid", cluster:2
Result: "admin.", cluster:2
Result: "blue-collar", cluster:6
Result: "management", cluster:4
Result: "services", cluster:9
Result: "management", cluster:2
Result: "entrepreneur", cluster:4
Result: "blue-collar", cluster

Result: "technician", cluster:2
Result: "entrepreneur", cluster:2
Result: "blue-collar", cluster:4
Result: "retired", cluster:6
Result: "admin.", cluster:0
Result: "admin.", cluster:2
Result: "technician", cluster:2
Result: "management", cluster:4
Result: "blue-collar", cluster:2
Result: "admin.", cluster:2
Result: "technician", cluster:6
Result: "blue-collar", cluster:2
Result: "blue-collar", cluster:6
Result: "blue-collar", cluster:6
Result: "admin.", cluster:4
Result: "management", cluster:9
Result: "admin.", cluster:6
Result: "entrepreneur", cluster:4
Result: "blue-collar", cluster:6
Result: "blue-collar", cluster:2
Result: "admin.", cluster:6
Result: "blue-collar", cluster:2
Result: "blue-collar", cluster:2
Result: "blue-collar", cluster:2
Result: "blue-collar", cluster:2
Result: "admin.", cluster:4
Result: "entrepreneur", cluster:4
Result: "technician", cluster:6
Result: "services", cluster:6
Result: "blue-collar", cluster:6
Result: "unemployed", cluster:2
Result: "services", clu

Result: "blue-collar", cluster:2
Result: "services", cluster:2
Result: "entrepreneur", cluster:2
Result: "services", cluster:6
Result: "technician", cluster:2
Result: "blue-collar", cluster:0
Result: "housemaid", cluster:6
Result: "admin.", cluster:2
Result: "admin.", cluster:2
Result: "services", cluster:2
Result: "blue-collar", cluster:2
Result: "entrepreneur", cluster:2
Result: "blue-collar", cluster:9
Result: "management", cluster:6
Result: "blue-collar", cluster:6
Result: "admin.", cluster:4
Result: "entrepreneur", cluster:2
Result: "services", cluster:6
Result: "unknown", cluster:2
Result: "blue-collar", cluster:2
Result: "blue-collar", cluster:6
Result: "blue-collar", cluster:2
Result: "retired", cluster:2
Result: "admin.", cluster:2
Result: "technician", cluster:2
Result: "admin.", cluster:2
Result: "blue-collar", cluster:6
Result: "admin.", cluster:4
Result: "technician", cluster:2
Result: "blue-collar", cluster:6
Result: "admin.", cluster:2
Result: "management", cluster:2
Res

Result: "blue-collar", cluster:6
Result: "self-employed", cluster:2
Result: "blue-collar", cluster:6
Result: "admin.", cluster:4
Result: "blue-collar", cluster:2
Result: "blue-collar", cluster:4
Result: "services", cluster:6
Result: "admin.", cluster:0
Result: "self-employed", cluster:6
Result: "blue-collar", cluster:2
Result: "retired", cluster:0
Result: "blue-collar", cluster:6
Result: "blue-collar", cluster:2
Result: "services", cluster:2
Result: "blue-collar", cluster:6
Result: "technician", cluster:6
Result: "blue-collar", cluster:6
Result: "blue-collar", cluster:2
Result: "blue-collar", cluster:4
Result: "blue-collar", cluster:6
Result: "admin.", cluster:6
Result: "self-employed", cluster:2
Result: "technician", cluster:2
Result: "self-employed", cluster:2
Result: "blue-collar", cluster:2
Result: "technician", cluster:4
Result: "self-employed", cluster:6
Result: "self-employed", cluster:4
Result: "blue-collar", cluster:6
Result: "self-employed", cluster:6
Result: "self-employed",

...

Result: "technician", cluster:2
Result: "technician", cluster:2
Result: "services", cluster:6
Result: "unknown", cluster:2
Result: "services", cluster:2
Result: "admin.", cluster:2
Result: "admin.", cluster:2
Result: "self-employed", cluster:2
Result: "entrepreneur", cluster:2
Result: "management", cluster:2
Result: "blue-collar", cluster:2
Result: "management", cluster:2
Result: "technician", cluster:6
Result: "admin.", cluster:6
Result: "management", cluster:2
Result: "retired", cluster:2
Result: "management", cluster:6
Result: "admin.", cluster:2
Result: "housemaid", cluster:2
Result: "technician", cluster:2
Result: "blue-collar", cluster:2
Result: "retired", cluster:2
Result: "admin.", cluster:6
Result: "technician", cluster:2
Result: "entrepreneur", cluster:2
Result: "housemaid", cluster:2
Result: "housemaid", cluster:4
Result: "unemployed", cluster:6
Result: "entrepreneur", cluster:2
Result: "services", cluster:2
Result: "blue-collar", cluster:2
Result: "blue-collar", cluster:5
R

Result: "admin.", cluster:6
Result: "technician", cluster:2
Result: "admin.", cluster:6
Result: "self-employed", cluster:6
Result: "blue-collar", cluster:2
Result: "services", cluster:4
Result: "technician", cluster:0
Result: "technician", cluster:6
Result: "blue-collar", cluster:2
Result: "admin.", cluster:2
Result: "admin.", cluster:6
Result: "admin.", cluster:6
Result: "technician", cluster:2
Result: "admin.", cluster:2
Result: "retired", cluster:2
Result: "entrepreneur", cluster:2
Result: "unemployed", cluster:2
Result: "technician", cluster:4
Result: "admin.", cluster:2
Result: "blue-collar", cluster:9
Result: "technician", cluster:2
Result: "unemployed", cluster:2
Result: "admin.", cluster:6
Result: "technician", cluster:2
Result: "housemaid", cluster:2
Result: "blue-collar", cluster:2
Result: "blue-collar", cluster:2
Result: "admin.", cluster:2
Result: "admin.", cluster:2
Result: "admin.", cluster:6
Result: "self-employed", cluster:2
Result: "admin.", cluster:6
Result: "unemploy

Result: "blue-collar", cluster:0
Result: "blue-collar", cluster:1
Result: "blue-collar", cluster:1
Result: "admin.", cluster:1
Result: "services", cluster:3
Result: "entrepreneur", cluster:0
Result: "blue-collar", cluster:9
Result: "blue-collar", cluster:1
Result: "blue-collar", cluster:3
Result: "student", cluster:3
Result: "blue-collar", cluster:1
Result: "blue-collar", cluster:4
Result: "admin.", cluster:1
Result: "blue-collar", cluster:1
Result: "admin.", cluster:1
Result: "services", cluster:3
Result: "management", cluster:3
Result: "services", cluster:3
Result: "blue-collar", cluster:1
Result: "management", cluster:1
Result: "services", cluster:3
Result: "services", cluster:3
Result: "technician", cluster:3
Result: "management", cluster:3
Result: "services", cluster:3
Result: "blue-collar", cluster:4
Result: "admin.", cluster:3
Result: "services", cluster:3
Result: "admin.", cluster:3
Result: "technician", cluster:4
Result: "technician", cluster:3
Result: "management", cluster:3


Result: "blue-collar", cluster:3
Result: "blue-collar", cluster:4
Result: "blue-collar", cluster:1
Result: "entrepreneur", cluster:0
Result: "technician", cluster:0
Result: "services", cluster:9
Result: "admin.", cluster:1
Result: "admin.", cluster:1
Result: "entrepreneur", cluster:1
Result: "admin.", cluster:1
Result: "self-employed", cluster:0
Result: "services", cluster:3
Result: "admin.", cluster:1
Result: "admin.", cluster:3
Result: "blue-collar", cluster:3
Result: "management", cluster:1
Result: "blue-collar", cluster:3
Result: "admin.", cluster:3
Result: "technician", cluster:1
Result: "housemaid", cluster:1
Result: "admin.", cluster:3
Result: "blue-collar", cluster:3
Result: "blue-collar", cluster:1
Result: "management", cluster:4
Result: "technician", cluster:1
Result: "services", cluster:3
Result: "blue-collar", cluster:4
Result: "services", cluster:1
Result: "services", cluster:8
Result: "blue-collar", cluster:1
Result: "services", cluster:1
Result: "services", cluster:9
Res

Result: "admin.", cluster:3
Result: "student", cluster:1
Result: "admin.", cluster:3
Result: "technician", cluster:1
Result: "admin.", cluster:1
Result: "unemployed", cluster:1
Result: "technician", cluster:5
Result: "unemployed", cluster:5
Result: "student", cluster:1
Result: "student", cluster:5
Result: "admin.", cluster:5
Result: "admin.", cluster:1
Result: "retired", cluster:3
Result: "management", cluster:4
Result: "admin.", cluster:1
Result: "technician", cluster:1
Result: "retired", cluster:5
Result: "retired", cluster:3
Result: "admin.", cluster:5
Result: "admin.", cluster:3
Result: "unknown", cluster:3
Result: "unknown", cluster:4
Result: "technician", cluster:5
Result: "technician", cluster:1
Result: "student", cluster:8
Result: "student", cluster:1
Result: "student", cluster:7
Result: "retired", cluster:1
Result: "services", cluster:1
Result: "admin.", cluster:1
Result: "retired", cluster:5
Result: "management", cluster:1
Result: "technician", cluster:4
Result: "technician",

In [33]:
syms

array(['"job"', '"housemaid"', '"services"', ..., '"retired"',
       '"technician"', '"retired"'], dtype='<U21')

In [34]:
X = np.genfromtxt('bank-additional-full.csv', dtype=object, delimiter=',')[:, 2:]

In [39]:
customer.loc[0]

0              56
1       housemaid
2         married
3        basic.4y
4              no
5              no
6              no
7       telephone
8             may
9             mon
10            261
11              1
12            999
13              0
14    nonexistent
15            1.1
16         93.994
17          -36.4
18          4.857
19           5191
Name: 0, dtype: object

In [41]:
len(X)

41189